##  Pandera and how to integrate it with Pandas
Introduction to Pandera
Pandera is a Python library for data validation that integrates seamlessly with Pandas. It allows you to define validation schemes declaratively and perform checks on DataFrames and Series.

Key Features
1. Pandas Integration: Works directly with DataFrames and Series
2. Declarative Validation: Defines rules clearly and legibly
3. Data Types: Supports validation of native Python and Pandas data types
4. Custom Validation: Allows you to create custom rules
5. Error Messages: Provides detailed messages about validation failures

In [1]:
pip install pandera

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------------ --- 1.8/2.0 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 8.6 MB/s  0:00:00

   ----- ---------------------------------- 1/8 [typeguard]
   ---------- ----------------------------- 2/8 [pydantic-core]
   -------------------- ------------------- 4/8 [annotated-types]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]
   ------------------------------ --------- 6/8 [pydantic]


In [ ]:
import pandas as pd
import pandera as pa

# Creating a sample DataFrame
df = pd.DataFrame({
    'age': [25, 30, 35],
    'salary': [5000, 6000, 7000  ],
    'email': ['joao@email.com', 'maria@email.com', 'pedro@email.com']
})

# Defining the validation schema
schema = pa.DataFrameSchema({
    'age': pa.Column(int, checks=pa.Check.ge(0)),  # age >= 0
    'salary': pa.Column(float, checks=pa.Check.gt(0)),  # salary > 0
    'email': pa.Column(str, checks=pa.Check.str_matches(r'^[^@]+@[^@]+\.[^@]+$'))
})

# Validating the DataFrame
try:
    schema.validate(df)
    print("Valid data!")
except pa.errors.SchemaError as e:
    print(f"Validation error: {e}")


Validation error: expected series 'salary' to have type float64, got int64


C:\Users\eduardo.siqueira\AppData\Local\miniconda3\envs\dados\Lib\site-packages\pandera\_pandas_deprecated.py:149: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


# Supported Validation Types
Column Validation:

- Data type
- Null values
- Unique values
- Values ​​in a specific list
- Row Validation:

- Relationships between columns
- Aggregations
- Complex conditions
- DataFrame Validation:

- DataFrame structure
- Number of rows/columns
- Indexes

# Advanced Validation Example

In [4]:
# More complex schema
advanced_schema = pa.DataFrameSchema({
    'age': pa.Column(
        int,
        checks=[
            pa.Check.ge(0),   # Age must be greater than or equal to zero
            pa.Check.le(120)  # Age must be less than or equal to 120
        ],
        nullable=False  # Null values are not allowed
    ),
    'salary': pa.Column(
        float,
        checks=[
            pa.Check.gt(0),        # Salary must be greater than zero
            pa.Check.lt(1_000_000) # Salary must be less than 1 million
        ]
    ),
    'email': pa.Column(
        str,
        checks=pa.Check.str_matches(r'^[^@]+@[^@]+\.[^@]+$'),  # Validates email format
        nullable=False  # Null values are not allowed
    ),
    'department': pa.Column(
        str,
        checks=pa.Check.isin(['IT', 'HR', 'Sales', 'Finance'])  # Department must be in this allowed list
    )
})

# Validation with transformation
def adjust_salary(df):
    df = df.copy()
    df['salary'] = df['salary'] * 1.1  # 10% increase
    return df

# Using pipe for transformation
schema_with_transformation = pa.DataFrameSchema({
    'salary': pa.Column(
        float,
        checks=pa.Check.gt(0),
        coerce=True
    )
})

# Example usage:
validated_df = schema_with_transformation.validate(df)  # Checks if the column is valid
transformed_df = validated_df.pipe(adjust_salary)       # Applies the transformation


In [5]:
validated_df

,age,salary,email
0,25,5000.0,joao@email.com
1,30,6000.0,maria@email.com
2,35,7000.0,pedro@email.com


In [6]:
transformed_df

,age,salary,email
0,25,5500.0,joao@email.com
1,30,6600.0,maria@email.com
2,35,7700.0,pedro@email.com
